In [79]:
import os
import pandas as pd

import warnings
warnings.filterwarnings('ignore')


In [42]:
# import data
patient_labeled = pd.read_csv(r'C:\Users\rodge\OneDrive\Desktop\SML\Group Project\comp90051\data\processedData\trainData_label.csv')

folder_path = r'C:\Users\rodge\OneDrive\Desktop\SML\Group Project\comp90051\data\rawData\unzip_file'

# Read all the csv files in the folder
for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file)
    
    if file.endswith('.csv'):
        file_name = os.path.splitext(file)[0]
        globals()[file_name] = pd.read_csv(file_path)

C:\Users\rodge\AppData\Local\Temp\ipykernel_35436\2882258290.py:12: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[file_name] = pd.read_csv(file_path)


In [43]:
def true_records (data,pat_id):
   
    data1 = data.merge(pat_id[['LOG_ID', 'MRN']], on='LOG_ID', how='inner', suffixes=('data', 'pat_id'))
    data_filtered= data1[data1['MRNdata'] == data1['MRNpat_id']]
    
    data_filtered = data_filtered.drop(columns=['MRNpat_id'])
    data_filtered = data_filtered.rename(columns={'MRNdata': 'MRN'})
    data2 = data_filtered.drop_duplicates()

    return data2

In [44]:
patient_labeled.head()

,LOG_ID,MRN,Cardiovascular
0,8c615f6805325d06,692072abc16db15d,1.0
1,0ecf38ed306338f6,4c29aee08612d793,1.0
2,595439a55deda0d5,71d77695a69b26d2,1.0
3,36662c223daeac3a,08b0d6a24cf382ad,1.0
4,1383c875d3b6ffbd,4c29aee08612d793,1.0


### Check patient's medicial history based on ICD-10 code

In [86]:

patient_info = true_records(patient_information, pat_id)
patient_history = patient_history.rename(columns={'mrn': 'MRN'})

keyword = ['heart','coronary','Hypotension','hypertension','Arrhythmia','cardiac','myocardial']

patient_mrn = patient_labeled[['MRN']]

patient_with_icdCode = pd.merge(patient_mrn, patient_history, on='MRN', how='left')
patient_with_icdCode = patient_with_icdCode.drop_duplicates()
patient_with_heart_disease = patient_with_icdCode[patient_with_icdCode['dx_name'].str.contains('|'.join(keyword), case=False, na=False)]

patient_with_heart_disease = patient_with_heart_disease[['MRN']]
patient_with_heart_disease = patient_with_heart_disease.drop_duplicates()
patient_with_heart_disease['cardiac_history'] = 1
patient_with_heart_disease.head()
print(patient_with_heart_disease.shape)


(322, 2)


In [53]:
patient_add_cardiac = pd.merge(patient_labeled, patient_with_heart_disease, on='MRN', how='left')
patient_add_cardiac['cardiac_history'] = patient_add_cardiac['cardiac_history'].fillna(0)
patient_add_cardiac.head()


,LOG_ID,MRN,Cardiovascular,cardiac_history
0,8c615f6805325d06,692072abc16db15d,1.0,1.0
1,0ecf38ed306338f6,4c29aee08612d793,1.0,0.0
2,595439a55deda0d5,71d77695a69b26d2,1.0,0.0
3,36662c223daeac3a,08b0d6a24cf382ad,1.0,1.0
4,1383c875d3b6ffbd,4c29aee08612d793,1.0,0.0


### Check patient lipid lab results

In [65]:
patient_id = patient_labeled[['LOG_ID','MRN']]
patient_labs = pd.merge(patient_labs, patient_id, on=['LOG_ID','MRN'], how='left')

lipid_keywords = ['lipid','cholesterol','lipoprotein','triglycerides','LDL','HDL']

patient_labs_lipid = patient_labs[patient_labs['Lab Name'].str.contains('|'.join(lipid_keywords), case=False, na=False)]

lipid_lab_list = patient_labs_lipid['Lab Name'].unique()





In [72]:
patient_labs_lipid = patient_labs_lipid[['LOG_ID','MRN','Lab Name','Abnormal Flag']]
patient_labs_lipid = patient_labs_lipid.drop_duplicates()
patient_labs_lipid

,LOG_ID,MRN,Lab Name,Abnormal Flag
1186,0b31b1841a16cfc5,2209857f3064e608,Cholesterol,N
1747,a361a18022b74d51,b6fa453612f4cf27,Cholesterol,N
2841,b9d864a970bbe767,efaf9b053b35e04c,Cholesterol.in LDL,N
3999,373354f4477cbe19,826e08205890682f,Cholesterol.in LDL,N
4556,a96b940dadfdf0d3,2209857f3064e608,Cholesterol.in LDL,N
...,...,...,...,...
29067671,fb55329489c3132a,dc99ac0ac89887aa,Cholesterol.in HDL,N
29071136,3307a482de81d9b0,0fa1548b20065781,Cholesterol.in HDL,L
29075398,6e7a961195d9a86b,ce87c080e249868a,Cholesterol.in HDL,L
29078768,2d369d64740b2ab5,68d00515360d5d71,Cholesterol.in HDL,L


In [81]:
def prioritize_abnormal_flag(series):
    flags = {'H': 3, 'L': 2, 'N': 1, None: 0}
    max_index = series.map(flags).idxmax()

    if pd.isnull(max_index):
        return None
    else:


        return series.loc[max_index]

reshaped = patient_labs_lipid.pivot_table(
    index=['LOG_ID', 'MRN'],
    columns='Lab Name',
    values='Abnormal Flag',
    aggfunc=prioritize_abnormal_flag,
    fill_value=None  
)

patient_lipid_result = reshaped.drop_duplicates()
patient_lipid_result.head()


,Lab Name,Apolipoprotein A-I,Apolipoprotein B-100,Cholesterol,Cholesterol.in HDL,Cholesterol.in LDL,Cholesterol.in VLDL,Cholesterol.in lipoprotein (little a),Cholesterol.non HDL,Coagulation surface induced.hexagonal phase phospholipid,LDL 1,...,LDL 4,LDL 5,LDL 6,LDL 7,Lipoprotein (little a),Lipoprotein midband A,Lipoprotein midband B,Lipoprotein midband C,Lipoprotein.beta.subparticle,Lupus anticoagulant neutralization.dilute phospholipid
LOG_ID,MRN,,,,,,,,,,,,,,,,,,,,,
00045c7713e9679e,ea2a778e62bc9ce9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
0004678c0906bb9e,6b03d87f76b29311,NaN,NaN,N,L,N,N,NaN,N,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
002eacf7d7ced35a,aa49d9d8d79f1fe0,NaN,NaN,H,L,N,N,NaN,H,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
003b45420d1a61ab,9f4394a2c00533c9,NaN,NaN,N,N,N,N,NaN,N,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0044651456148b1d,86a10fac32e8d8e6,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
null_counts = patient_lipid_result.isna().sum()
print(null_counts)


Lab Name
Apolipoprotein A-I                                          67
Apolipoprotein B-100                                        64
Cholesterol                                                  9
Cholesterol.in HDL                                          14
Cholesterol.in LDL                                          13
Cholesterol.in VLDL                                         14
Cholesterol.in lipoprotein (little a)                       57
Cholesterol.non HDL                                         14
Coagulation surface induced.hexagonal phase phospholipid    61
LDL 1                                                       71
LDL 2                                                       71
LDL 3                                                       71
LDL 4                                                       71
LDL 5                                                       71
LDL 6                                                       71
LDL 7                                         

There are too much null value for lipid labs, so we decide do not add these features into the trainning dataset.

## Check labs relates to Blood routine

In [83]:
bloodTest_keywords = ['hemoglobin','white blood cells','wbc','red blood cells','rbc']
patient_labs_blood = patient_labs[patient_labs['Lab Name'].str.contains('|'.join(bloodTest_keywords), case=False, na=False)]
patient_labs_blood = patient_labs_blood[['LOG_ID','MRN','Lab Name','Abnormal Flag']]
patient_labs_blood = patient_labs_blood.drop_duplicates()

def prioritize_abnormal_flag(series):
    flags = {'H': 3, 'L': 2, 'N': 1, None: 0}
    max_index = series.map(flags).idxmax()

    if pd.isnull(max_index):
        return None
    else:


        return series.loc[max_index]

reshaped = patient_labs_blood.pivot_table(
    index=['LOG_ID', 'MRN'],
    columns='Lab Name',
    values='Abnormal Flag',
    aggfunc=prioritize_abnormal_flag,
    fill_value=None  
)

patient_blood_result = reshaped.drop_duplicates()
patient_blood_result.head()


,Lab Name,Carboxyhemoglobin/Hemoglobin.total,Erythrocyte mean corpuscular hemoglobin,Erythrocyte mean corpuscular hemoglobin concentration,Hemoglobin,Hemoglobin A/Hemoglobin.total,Hemoglobin A1c/Hemoglobin.total,Hemoglobin A2/Hemoglobin.total,Hemoglobin C/Hemoglobin.total,Hemoglobin D/Hemoglobin.total,Hemoglobin E/Hemoglobin.total,Hemoglobin F/Hemoglobin.total,Hemoglobin H inclusion bodies,Hemoglobin S,Hemoglobin S/Hemoglobin.total,Hemoglobin.free,Hemoglobin.gastrointestinal.lower,Methemoglobin,Methemoglobin/Hemoglobin.total
LOG_ID,MRN,,,,,,,,,,,,,,,,,,
0001c7ea6b8a8c7a,d18eebe95e2c4292,NaN,N,N,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0003ec6d34d9ba20,0d63f05df441b4bc,NaN,N,N,L,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0004678c0906bb9e,6b03d87f76b29311,NaN,L,N,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000a6aa2d4aab6bd,897611427f9fc4f7,NaN,L,L,L,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000d639a04b86624,358da6bb6f5e4836,NaN,H,N,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
patient_blood_result

,Lab Name,Carboxyhemoglobin/Hemoglobin.total,Erythrocyte mean corpuscular hemoglobin,Erythrocyte mean corpuscular hemoglobin concentration,Hemoglobin,Hemoglobin A/Hemoglobin.total,Hemoglobin A1c/Hemoglobin.total,Hemoglobin A2/Hemoglobin.total,Hemoglobin C/Hemoglobin.total,Hemoglobin D/Hemoglobin.total,Hemoglobin E/Hemoglobin.total,Hemoglobin F/Hemoglobin.total,Hemoglobin H inclusion bodies,Hemoglobin S,Hemoglobin S/Hemoglobin.total,Hemoglobin.free,Hemoglobin.gastrointestinal.lower,Methemoglobin,Methemoglobin/Hemoglobin.total
LOG_ID,MRN,,,,,,,,,,,,,,,,,,
0001c7ea6b8a8c7a,d18eebe95e2c4292,NaN,N,N,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0003ec6d34d9ba20,0d63f05df441b4bc,NaN,N,N,L,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0004678c0906bb9e,6b03d87f76b29311,NaN,L,N,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000a6aa2d4aab6bd,897611427f9fc4f7,NaN,L,L,L,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000d639a04b86624,358da6bb6f5e4836,NaN,H,N,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
f6a418d204620ce8,593f30247cbd69e4,NaN,N,N,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN
f75de9e59730787a,f23adb9fcf82b8a3,N,L,H,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
f8fd22dda7d11e8b,9ecdc4b1d0ae897a,L,N,N,N,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN


In [84]:
null_counts = patient_blood_result.isna().sum()
print(null_counts)

Lab Name
Carboxyhemoglobin/Hemoglobin.total                       142
Erythrocyte mean corpuscular hemoglobin                    8
Erythrocyte mean corpuscular hemoglobin concentration      8
Hemoglobin                                                 8
Hemoglobin A/Hemoglobin.total                            360
Hemoglobin A1c/Hemoglobin.total                          154
Hemoglobin A2/Hemoglobin.total                           359
Hemoglobin C/Hemoglobin.total                            360
Hemoglobin D/Hemoglobin.total                            361
Hemoglobin E/Hemoglobin.total                            364
Hemoglobin F/Hemoglobin.total                            356
Hemoglobin H inclusion bodies                            364
Hemoglobin S                                             382
Hemoglobin S/Hemoglobin.total                            367
Hemoglobin.free                                          369
Hemoglobin.gastrointestinal.lower                        338
Methemoglobin  

The result shows that only 23% of patients have lab results relates to the blood routine which is too small.

### Check what labs have done by most of the patients in the dataset


In [100]:
patient_labs = pd.merge(patient_labs, patient_id, on=['LOG_ID','MRN'], how='inner')
patient_labs = patient_labs[['LOG_ID','MRN','Lab Name','Abnormal Flag']]
patient_labs.drop_duplicates()


def prioritize_abnormal_flag(series):
    flags = {'H': 3, 'L': 2, 'N': 1, None: 0}
    max_index = series.map(flags).idxmax()

    if pd.isnull(max_index):
        return None
    else:


        return series.loc[max_index]

result_df = patient_labs.pivot_table(
    index=['LOG_ID', 'MRN', 'Lab Name'],
    values='Abnormal Flag',
    aggfunc=prioritize_abnormal_flag,
    fill_value=None
).reset_index()

patient_lab_summary = result_df.copy()
patient_lab_summary.sort_values(by=['LOG_ID', 'Lab Name'], inplace=True)
patient_lab_summary

,LOG_ID,MRN,Lab Name,Abnormal Flag
0,00cd402635d98736,07ed75a59473390e,Alanine aminotransferase,N
1,00cd402635d98736,07ed75a59473390e,Albumin,N
2,00cd402635d98736,07ed75a59473390e,Alkaline phosphatase,N
3,00cd402635d98736,07ed75a59473390e,Anion gap,N
4,00cd402635d98736,07ed75a59473390e,Aspartate aminotransferase,N
...,...,...,...,...
63559,ffd11a243a46841a,e55867208219e1de,Urinalysis specimen collection method,N
63560,ffd11a243a46841a,e55867208219e1de,Urobilinogen,N
63561,ffd11a243a46841a,e55867208219e1de,Varicella zoster virus Ab.IgG,H
63562,ffd11a243a46841a,e55867208219e1de,Varicella zoster virus Ab.IgM,N


In [115]:
lab_counts = patient_lab_summary['Lab Name'].value_counts().sort_values(ascending=False)
top_25_labs = lab_counts.head(25)

lab_names_array = top_25_labs.index.tolist()

filtered_data = patient_lab_summary[patient_lab_summary['Lab Name'].isin(lab_names_array)]

non_null_counts = filtered_data.groupby('Lab Name')['Abnormal Flag'].count().sort_values(ascending=False)
non_null_counts



Lab Name
Glucose                                                     952
Potassium                                                   952
Sodium                                                      950
Carbon dioxide                                              949
Hemoglobin                                                  944
Creatinine                                                  942
Chloride                                                    942
Urea nitrogen                                               942
Calcium                                                     940
Erythrocytes                                                937
Anion gap                                                   935
Glomerular filtration rate/1.73 sq M.predicted.non black    935
Glomerular filtration rate/1.73 sq M.predicted.black        935
Hematocrit                                                  934
Erythrocyte mean corpuscular hemoglobin concentration       933
Erythrocyte mean corpuscular he

Based on the result, we can identify some features that are correlated to cardiac disease such as Hemoglobin, Calcium, Erythrocytes, Hematocrit, Erythrocyte mean corpuscular hemoglobin concentration,Erythrocyte mean corpuscular hemoglobin,Erythrocyte distribution width,Erythrocyte mean corpuscular volume.

In [119]:
heart_related_keywords = ['Hemoglobin','Calcium','Erythrocytes','Hematocrit','Erythrocytes','Erythrocyte mean corpuscular hemoglobin concentration',
                                'Erythrocyte mean corpuscular hemoglobin','Erythrocyte distribution width','Erythrocyte mean corpuscular volume']

patient_labs_heart = patient_labs[patient_labs['Lab Name'].isin(heart_related_keywords)]

result_df = patient_labs_heart.pivot_table(
    index=['LOG_ID', 'MRN'],
    columns='Lab Name',
    values='Abnormal Flag',
    aggfunc=prioritize_abnormal_flag,
    fill_value=None
).reset_index()

heart_related_features = result_df.drop_duplicates()
heart_related_features

Lab Name,LOG_ID,MRN,Calcium,Erythrocyte distribution width,Erythrocyte mean corpuscular hemoglobin,Erythrocyte mean corpuscular hemoglobin concentration,Erythrocyte mean corpuscular volume,Erythrocytes,Hematocrit,Hemoglobin
0,00cd402635d98736,07ed75a59473390e,N,N,N,N,N,N,N,N
1,011e00acde3a3342,4a972c7f6056b8c8,N,N,N,N,N,N,N,N
2,012721471b0a176d,1821f1494d13a720,L,H,L,N,L,N,L,L
3,0154bf65c2e2653c,d00bb637ec44bf76,N,H,N,N,N,N,L,L
4,01dc302bc8104d42,40126677abeddb56,L,H,N,H,N,L,L,L
...,...,...,...,...,...,...,...,...,...,...
957,fe9324aa46382ff5,a1fcee88b4a59390,L,H,N,N,H,L,L,L
958,ff5c0ba0edf25cf2,8692432b64cd5c7e,L,N,N,N,N,H,L,L
959,ffaf0d4ee71c511c,068ef29060e6b274,L,N,N,N,N,N,N,N
960,ffcb585a8a103c92,6163484df7b3e7ba,N,N,N,N,N,L,L,L


In [122]:
# Merge the data with heart related features
patient_heart_feature = pd.merge(patient_labeled, heart_related_features, on=['LOG_ID', 'MRN'], how='left')
patient_heart_feature_filled = patient_heart_feature.fillna(0)
patient_heart_feature_filled.head()

,LOG_ID,MRN,Cardiovascular,Calcium,Erythrocyte distribution width,Erythrocyte mean corpuscular hemoglobin,Erythrocyte mean corpuscular hemoglobin concentration,Erythrocyte mean corpuscular volume,Erythrocytes,Hematocrit,Hemoglobin
0,8c615f6805325d06,692072abc16db15d,1.0,0,0,0,0,0,0,0,0
1,0ecf38ed306338f6,4c29aee08612d793,1.0,0,0,0,0,0,0,0,0
2,595439a55deda0d5,71d77695a69b26d2,1.0,L,H,N,L,H,H,L,L
3,36662c223daeac3a,08b0d6a24cf382ad,1.0,L,H,N,H,N,L,L,L
4,1383c875d3b6ffbd,4c29aee08612d793,1.0,0,0,0,0,0,0,0,0


### Merge with the features that have extracted from the other sections

In [126]:
features_part2 = pd.merge(patient_heart_feature_filled, patient_add_cardiac, on=['LOG_ID', 'MRN','Cardiovascular'], how='inner')
print(features_part2.shape)
features_part2.head()

(1604, 12)


,LOG_ID,MRN,Cardiovascular,Calcium,Erythrocyte distribution width,Erythrocyte mean corpuscular hemoglobin,Erythrocyte mean corpuscular hemoglobin concentration,Erythrocyte mean corpuscular volume,Erythrocytes,Hematocrit,Hemoglobin,cardiac_history
0,8c615f6805325d06,692072abc16db15d,1.0,0,0,0,0,0,0,0,0,1.0
1,0ecf38ed306338f6,4c29aee08612d793,1.0,0,0,0,0,0,0,0,0,0.0
2,595439a55deda0d5,71d77695a69b26d2,1.0,L,H,N,L,H,H,L,L,0.0
3,36662c223daeac3a,08b0d6a24cf382ad,1.0,L,H,N,H,N,L,L,L,1.0
4,1383c875d3b6ffbd,4c29aee08612d793,1.0,0,0,0,0,0,0,0,0,0.0


In [127]:
folder_path = r'C:\Users\rodge\OneDrive\Desktop\SML\Group Project\comp90051\data\processedData'
file_name = 'features_part2.csv'

full_path = f'{folder_path}\\{file_name}'
features_part2.to_csv(full_path, index=False)